In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
import seaborn as sns
import matplotlib.pyplot  as plt
import sklearn
import plotly.express as px
import joblib

In [3]:
df=pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


# **Preprocessing**

In [4]:
#convert any class 0 to 1 and any class 1 to -1
df.Class=df.Class.map({0:1,1:-1})

In [5]:
#divide data into normal and fraud
df_normal=df[df.Class==1]
df_fraud=df[df.Class==-1]

x_normal=df_normal.drop("Class",axis=1)
x_fraud=df_fraud.drop("Class",axis=1)

y_normal=df_normal.Class
y_fraud=df_fraud.Class

In [6]:
(x_normal.shape,x_fraud.shape)

((284315, 30), (492, 30))

In [7]:
from sklearn.preprocessing import StandardScaler 

from sklearn.model_selection import cross_validate ,cross_val_score

from sklearn.ensemble import IsolationForest

In [8]:
scaler=StandardScaler()
x_normal=scaler.fit_transform(x_normal)
x_fraud=scaler.transform(x_fraud)

# **Model**

In [9]:
model=IsolationForest(random_state=0)
model.fit(x_normal)

IsolationForest(random_state=0)

In [10]:
y_train_predict=model.predict(x_normal)

In [11]:
np.unique(y_train_predict)

array([-1,  1])

In [12]:
data_train=pd.DataFrame({"y_train":y_normal,
                   "y_train_predict":y_train_predict})

In [13]:
data_train["y_train_predict"].value_counts()

y_train_predict
 1    274286
-1     10029
Name: count, dtype: int64

In [14]:
y_test_predict=model.predict(x_fraud)

In [15]:
np.unique(y_train_predict)

array([-1,  1])

In [16]:
data_test=pd.DataFrame({"y_test":y_fraud,
                   "y_test_predict":y_test_predict})

In [17]:
data_test["y_test_predict"].value_counts()

y_test_predict
-1    415
 1     77
Name: count, dtype: int64

In [18]:
data_test["y_test"].value_counts()

y_test
-1    492
Name: count, dtype: int64

# **Model evalation**

In [19]:
from sklearn.metrics import f1_score,recall_score

In [20]:
train_score=f1_score(y_normal,y_train_predict)
test_score=f1_score(y_fraud,y_test_predict,pos_label=-1)

print(f"F1 Score of train = {train_score}")
print(f"F1 Score of test = {test_score}")

F1 Score of train = 0.9820462190364858
F1 Score of test = 0.9151047409040793


In [21]:
train_recall=recall_score(y_normal,y_train_predict)
test_recall=recall_score(y_fraud,y_test_predict,pos_label=-1)

print(f"recall Score of train = {train_recall}")
print(f"recall Score of test = {test_recall}")

recall Score of train = 0.9647257443328703
recall Score of test = 0.8434959349593496


# **Save model**

In [22]:
import joblib
joblib.dump(model,"model.pkl")

['model.pkl']